# Introduction

In this notebook I explore using my MP-Iteration to solve the problem given as equation (4.1) in [This Niall Madden Paper](https://jadler.math.tufts.edu/publications/files/2015_SingPert_IMAJNA.pdf).

Note that I make use $$ M = I + \alpha \nabla u \nabla u^T.$$

# Results

The meshes produced in this notebook are not what we desire. They put too many mesh points along the center of the axes and miss the origin (miss the curvature of the solution). These results are one of the big reasons we start experimenting with Hessian based monitior functions. On a positive note, the paraview plots from this notebook show that orthogonality is being satisfied in the mesh PDE and that the mesh is properly adapting to the chosen M - the M is just bad!

# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install firedrake

# hide output
%%capture

try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [3]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [4]:
# import firedrake tools

from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d



# The Known Solution

In [5]:
def known_solution(x, epsilon):


  eps = Constant(epsilon)

  parta = cos((pi/2)*x[0]) - ( exp(-x[0]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )
  partb = 1 - x[1] - ( exp(-x[1]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )
  u_exact = parta * partb

  return u_exact

# Physical Solver

In [6]:

# phyiscal solver

def physical_solver(mesh, x, W, epsilon = 0.1, plots = True):

  # test and trial functions

  u_hat = TrialFunction(W)
  v_hat = TestFunction(W)


  # set f (rhs of pde)


  eps = Constant(epsilon)

  # f_hat = -eps**2*((-pi**2*cos(pi*x[0]/2)/4 - exp(-x[0]/eps)/(eps**2*(1 - exp(-1/eps))))*(-x[1] + 1 - (exp(-x[1]/eps) - exp(-1/eps))/(1 - exp(-1/eps))) - (cos(pi*x[0]/2) - (exp(-x[0]/eps) - exp(-1/eps))/(1 - exp(-1/eps)))*exp(-x[1]/eps)/(eps**2*(1 - exp(-1/eps)))) + (cos(pi*x[0]/2) - (exp(-x[0]/eps) - exp(-1/eps))/(1 - exp(-1/eps)))*(-x[1] + 1 - (exp(-x[1]/eps) - exp(-1/eps))/(1 - exp(-1/eps)))



  # The jacobian matrix of the mesh solution x

  J_F = grad(x) # jacobian matrix


  #pull components out
  x_xi = J_F[0,0]
  x_eta = J_F[0,1]
  y_xi = J_F[1,0]
  y_eta = J_F[1,1]

  # determinant
  det_J = x_xi * y_eta - x_eta * y_xi


  # inverse of the jacobian


  J_F_inv = inv(J_F)
  J_F_inv_T = transpose(J_F_inv)


  # compute the rhs of the pde

  a1 = (1/det_J) * as_vector([y_eta, -x_eta])
  a2 = (1/det_J) * as_vector([-y_xi, x_xi])

  grad_xi_sol = grad( known_solution(x, epsilon) )
  grad_x_sol = dot(J_F_inv_T, grad_xi_sol)

  val1 = inner( det_J * a1 , grad_x_sol )
  val2 = inner( det_J * a2 , grad_x_sol )

  div_term = (1/det_J) * div( as_vector([val1, val2]) )

  f_hat = -eps**2 * div_term + known_solution(x, epsilon)



  # compute grad_(x) of u (HR 3.8)

  grad_x_u = dot(J_F_inv_T, grad(u_hat))


  # compute grad_(x) of v (HR 3.8)

  grad_x_v = dot(J_F_inv_T, grad(v_hat))


  # The bilinear lhs
  # based on weak forms in Gockenbach and a change of variables

  a = ( (eps**2) * inner( grad_x_u, grad_x_v) + inner(u_hat, v_hat) ) * det_J * dx

  # The linear rhs

  L = (inner( f_hat, v_hat ) * det_J) * dx



  # define the BC's


  # bcy1 = (cos((pi/2)*0) - ( exp(-0/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )) * ( 1 - x[1] - ( exp(-x[1]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) ) )
  # bcy2 = (cos((pi/2)*1) - ( exp(-1/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )) * ( 1 - x[1] - ( exp(-x[1]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) ) )
  # bcx1 = (cos((pi/2)*x[0]) - ( exp(-x[0]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )) * ( 1 - 0 - ( exp(-0/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) ) )
  # bcx2 = (cos((pi/2)*x[0]) - ( exp(-x[0]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )) * ( 1 - 1 - ( exp(-1/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) ) )
  # bc1 = DirichletBC(W, bcy1 , 1) #left
  # bc2 = DirichletBC(W, bcy2 , 2) #right
  # bc3 = DirichletBC(W, bcx1 , 3) #bottom
  # bc4 = DirichletBC(W, bcx2 , 4) #top
  # bcs = [ bc1, bc2, bc3, bc4]


  bcs = DirichletBC(W, known_solution(x, epsilon), "on_boundary")


  # solve the problem

  sol = Function(W)


  solve(a == L, sol, bcs = bcs)


  # exact solution

  exact_sol = known_solution(x, epsilon)


  # L2 error

  phys_error_1 = norm( sqrt(det_J) * (exact_sol - sol) )

  phys_error_2 = norm( sqrt(det_J) * dot(J_F_inv_T ,grad(exact_sol - sol)))



  # compute the mesh density function
  # Mesh Density Matrix that depends on u'

  grad_u = dot(J_F_inv_T, grad(sol))
  alpha = 0.1

  M = Identity(2) + alpha * outer(grad_u, grad_u)


  M_func1 = Function(W).interpolate(M[0, 0]) # 1 + u_x^2
  M_func2 = Function(W).interpolate(M[0, 1]) # 1 + u_y^2
  M_func3 = Function(W).interpolate(M[1, 0]) # u_x * u_y
  M_func4 = Function(W).interpolate(M[1, 1]) # u_x * u_y


  # return the solution and the mesh density function


  return sol, M, phys_error_1, phys_error_2, M_func1, M_func2, M_func3,  M_func4

# Mesh Solver (Matrix Form w/o 1/|J|)

In [7]:
# mesh solver


def mesh_solver_noJ(xi, eta, M_matrix, x, V, deg = 1, rtol = 1e-8, stol = 1e-8, atol = 1e-50, maxit = 10, scale_BI = False):

  # WINSLOW BASED ON (6.131) in HR

  # function to hold mesh solution
  mesh_sol = Function(V)

  # Jacobian

  J_matrix = grad(mesh_sol)
  detJ = det(J_matrix)

  # JMJ part

  # The transformed nonlinear problem based on HR (3.8), (3.9) or (6.15), (6.131), (6.132, (6.133)
  JMJ_matrix = transpose(J_matrix) * M_matrix * J_matrix
  JMJ_inv = inv(JMJ_matrix)



  # create problem


  # test function

  v = TestFunction(V)


  # WEAK FORM (based on Gockenbach and my own proofs)

  pt1 = inner(transpose(grad(v)), detJ * JMJ_inv) * dx

  # Boundary Integrals

  Minv = inv(M_matrix)

  if scale_BI:

    print("I AM SCALING THE BI")
    print()

    scale = 1e15

    BI1 = scale * (- Minv[0, 1] * v[1] )*ds(1)
    BI2 = scale * ( Minv[0, 1] * v[1] )*ds(2)
    BI3 = scale * (- Minv[1, 0] * v[0] )*ds(3)
    BI4 = scale * ( Minv[1, 0] * v[0] )*ds(4)

  else:

    BI1 = (- Minv[0, 1] * v[1] )*ds(1)
    BI2 = ( Minv[0, 1] * v[1] )*ds(2)
    BI3 = (- Minv[1, 0] * v[0] )*ds(3)
    BI4 =  ( Minv[1, 0] * v[0] )*ds(4)



  BI = BI1 + BI2 + BI3 + BI4

  F = pt1 - BI


  # The initial guess (current (x,y) mesh)

  mesh_sol.interpolate(as_vector([x[0], x[1]]))


  # the Dirichlet bc's


  bc1 = DirichletBC(V.sub(0), Constant(0), 1) # x=0 on left side
  bc2 = DirichletBC(V.sub(0), Constant(1), 2) # x=1 on right side
  bc3 = DirichletBC(V.sub(1), Constant(0), 3) # y=0 on bottom
  bc4 = DirichletBC(V.sub(1), Constant(1), 4) # y=1 on top
  bcs = [bc1, bc2, bc3, bc4]



  # Solve, NewtonLS

  print("-"*125)
  print("Nonlinear Mesh Solve:")



  solve(F == 0, mesh_sol, bcs = bcs, \
          solver_parameters = {'snes_converged_reason': None,\
                              'snes_monitor': None,\
                              "snes_stol": stol,\
                              "snes_rtol": rtol,\
                              "snes_atol": atol,\
                              "snes_max_it": maxit})

  print("-"*125)
  print()
  print()



  return mesh_sol


# MP-Iteration Code

In [8]:
# MP - Iteration Code that uses differant mesh solve

def MP_iter_woJ(mesh, xi, eta, W, V, x, N, iter_count, epsilon, deg = 1, plot_phys = True, plot_mesh = True, rtol = 1e-8, stol = 1e-8, atol = 1e-50, maxit = 10, scale_BI = False, plotsM = False):


  # set up for the loop
  # list to save physical erros

  phys_l2_errors = []
  phys_grad_errors = []


  # list to save progress

  result_list = []


  # set the loop for the set number of MP iterations

  for i in range(iter_count):


    # physical solve

    phys_sol, M, phys_error_1, phys_error_2, M_func1, M_func2, M_func3, M_func4 = physical_solver(mesh, x, W, epsilon, plots = plot_phys)


    # save errors

    phys_l2_errors.append(phys_error_1)
    phys_grad_errors.append(phys_error_2)


    # save x and phys sol to results

    result_list.append([x, phys_sol])




    if (plotsM and i == 0): # I want to plot the first physical solution and M found

      # plot physical solve and M components

      # zoom in on M plots
      rect_mesh = RectangleMesh(int(N/4), N, 0.25, 1)
      rectV = FunctionSpace(rect_mesh, "CG", 1)
      M_func1_rect = Function(rectV).interpolate(M_func1)
      M_func2_rect = Function(rectV).interpolate(M_func2)
      M_func3_rect = Function(rectV).interpolate(M_func3)
      M_func4_rect = Function(rectV).interpolate(M_func4)

      # known soln
      sol_known = Function(W).interpolate(known_solution(x, epsilon))


      #plot
      custom_cmap = "GnBu"
      fig, axes = plt.subplots(1, 2, figsize = (14, 6))

      # Plot the physical solution on the second subplot
      c = tricontourf(phys_sol, axes=axes[0], cmap = custom_cmap)
      m = triplot(mesh, axes = axes[0])
      axes[0].set_title('Approx Physical Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0].set_xlabel(r"$\xi$")
      axes[0].set_ylabel(r"$\eta$")
      axes[0].set_aspect('equal')
      plt.colorbar(c, ax=axes[0])

      # Plot the known solution
      c = tricontourf(sol_known, axes=axes[1], cmap = custom_cmap)
      m = triplot(mesh, axes = axes[1])
      axes[1].set_title('Known Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1].set_xlabel(r"$\xi$")
      axes[1].set_ylabel(r"$\eta$")
      axes[1].set_aspect('equal')
      plt.colorbar(c, ax=axes[1])

      #show
      plt.tight_layout()
      plt.show()
      print()
      print()


      fig, axes = plt.subplots(2, 2, figsize = (24, 14))
      # Plot m00 on mesh
      cm = tricontourf(M_func1_rect, axes=axes[0, 0], cmap = custom_cmap)
      m = triplot(rect_mesh, axes = axes[0, 0])
      axes[0, 0].set_title('m00 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0, 0].set_xlabel(r"$\xi$")
      axes[0, 0].set_ylabel(r"$\eta$")
      #axes[2,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[0, 0])


      # Plot m01 on mesh
      cm = tricontourf(M_func2_rect, axes=axes[0, 1], cmap = custom_cmap)
      m = triplot(rect_mesh, axes = axes[0, 1])
      axes[0, 1].set_title('m01 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0, 1].set_xlabel(r"$\xi$")
      axes[0, 1].set_ylabel(r"$\eta$")
      #axes[3,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[0, 1])

      # Plot m10 on mesh
      cm = tricontourf(M_func3_rect, axes=axes[1, 0], cmap = custom_cmap)
      m = triplot(rect_mesh, axes = axes[1, 0])
      axes[1, 0].set_title('m10 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1, 0].set_xlabel(r"$\xi$")
      axes[1, 0].set_ylabel(r"$\eta$")
      #axes[3,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[1, 0])

      # Plot m11 on mesh
      cm = tricontourf(M_func4_rect, axes=axes[1, 1], cmap = custom_cmap)
      m = triplot(rect_mesh, axes = axes[1, 1])
      axes[1, 1].set_title('m11 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1, 1].set_xlabel(r"$\xi$")
      axes[1, 1].set_ylabel(r"$\eta$")
      #axes[3,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[1, 1])


      #show
      plt.tight_layout()
      plt.show()
      print()
      print()


    # Mesh Solve

    mesh_sol = mesh_solver_noJ(xi, eta, M, x, V, deg = deg, rtol = rtol, stol = stol, atol = atol, maxit = maxit, scale_BI = scale_BI)


    # analyize mesh convergence

    if i > 0:

      mesh_check = norm(mesh_sol - x)


    # UPDATE x

    x = mesh_sol


  # do a final physical solve on the last mesh

  phys_sol, M, phys_error_1, phys_error_2, M_func1, M_func2, M_func3, M_func4 = physical_solver(mesh, x, W, epsilon, plots = plot_phys)
  phys_l2_errors.append(phys_error_1)
  phys_grad_errors.append(phys_error_2)


  # save x and phys sol to results

  result_list.append([x, phys_sol])



  return result_list, x, phys_l2_errors, phys_grad_errors, phys_sol, M, M_func1, M_func2, M_func3, M_func4

# The Test

In [9]:
# Set My Parameters

epsilon = 0.5 # start at 0.2 if uniform!
N = 24
diagonal = "right"
quadrilateral = False
x_choice = 0 # 0 = identity function, 1 = map uniform to shishkin, 2 = map shishkin to uniform
xi_choice = 0 # 0 = uniform mesh, 1 = shishkin mesh
custom_cmap = "GnBu"
deg = 1
MPIter = 3
cont_steps = 25
# SNES stopping criteria
stol = 1e-8
rtol = 1e-8
atol = 1e-50
maxit = 50


# save all progress
all_results = []

# scaling the BI's?
scale_BI = False


# list to save the x's
x_list = []




# Set xi


if xi_choice == 0: # ci is uniform

  mesh = UnitSquareMesh(N, N, quadrilateral = quadrilateral, diagonal = diagonal)
  xi, eta = SpatialCoordinate(mesh)


if xi_choice == 1: # xi is shishkin

  epsm = 0.025
  beta = 0.99/np.sqrt(2)
  tau_a = 1/2
  tau_b = 2*epsm*(1/beta)*np.log(N)
  tau = min(tau_a, tau_b)

  hB = 2*tau / N
  hI = 2*(1-tau) / N


  part1 = np.arange(0, tau, hB)
  part2 = np.arange(tau, 1.0+hI, hI)

  # w_x = np.concatenate((part1, part2[:-1]))
  # w_y = np.concatenate((part1, part2[:-1]))
  w_x = np.concatenate((part1, part2))
  w_y = np.concatenate((part1, part2))

  mesh = TensorRectangleMesh(w_x, w_y, quadrilateral = quadrilateral, diagonal = diagonal)
  xi, eta = SpatialCoordinate(mesh)




# function spaces

W = FunctionSpace(mesh, "CG", deg)
V = VectorFunctionSpace(mesh, "CG", deg)


# set x


if x_choice == 0: # x is identity function

  x = Function(V).interpolate(as_vector([xi, eta]))

elif x_choice == 1: # x maps uniform to shihskin

  epsx = 0.025
  beta = 0.99/np.sqrt(2)
  tau_a = 1/2
  tau_b = 2*epsx*(1/beta)*np.log(N)
  tau = min(tau_a, tau_b)

  x_shish = conditional( xi < 0.5, 2*xi*tau, tau + 2*(1-tau)*(xi-1/2) )
  y_shish = conditional( eta < 0.5, 2*eta*tau, tau + 2*(1-tau)*(eta-1/2) )

  x = Function(V).interpolate(as_vector([x_shish, y_shish]))


elif x_choice == 2: # x maps shihskin to uniform

  x_shish = conditional( xi <= tau, (1/(2*tau))*xi, (1/(2*(1-tau)))*( xi + 1 - 2*tau) )
  y_shish = conditional( eta < tau, (1/(2*tau))*eta, (1/(2*(1-tau)))*( eta + 1 - 2*tau))

  x = Function(V).interpolate(as_vector([x_shish, y_shish]))


# evaluate our starting x function so that we can visualize

# solution data
old_mesh_vals = mesh.coordinates.dat.data
xsol = np.array(x.at(old_mesh_vals))[:,0]
ysol = np.array(x.at(old_mesh_vals))[:,1]

# create mesh
initial_phys_mesh = UnitSquareMesh(N, N, diagonal  = diagonal, quadrilateral = quadrilateral)
nx = len(xsol)
new_mesh_vals = np.zeros((nx, 2))
new_mesh_vals[:,0] = xsol
new_mesh_vals[:,1] = ysol
initial_phys_mesh.coordinates.dat.data[:] = new_mesh_vals


print()

#physical error list

physical_l2_e = []
physical_grad_e = []
figures = []
fig_for_later = []







# # Save the known solution on a fine mesh for each epsilon in a VTK file
# t = 0
# # my fine mesh
# fine_mesh = UnitSquareMesh(2*N, 2*N, diagonal  = diagonal, quadrilateral = quadrilateral)
# xf, yf = SpatialCoordinate(fine_mesh)
# W_fine = FunctionSpace(fine_mesh, "CG", 1)
# V_fine = VectorFunctionSpace(fine_mesh, "CG", 1)
# x_fine = SpatialCoordinate(fine_mesh)
# # file path to save the vtu
# file_path = '/content/drive/My Drive/Madden_plots/SmallerEps_N49_fine_solutions.pvd'
# outfile = VTKFile(file_path)
# t = 0
# fine_sol = Function(W_fine)

# for i in range(cont_steps):
#   # define a fine mesh for plotting the solution
#   fine_sol.interpolate(known_solution(x_fine, epsilon))
#   # save the current sol in the file
#   outfile.write(fine_sol, time = t)

#   epsilon = (9/10)*epsilon
#   t += 1





# Set the epsilon continuation + MP solves




for i in range(cont_steps):


  physical_l2_e.append(epsilon)
  physical_grad_e.append(epsilon)


  print("_"*60 + " epsilon = " + str(epsilon) + " " + "_"*60 )
  print()
  print()
  print()




  plotsM = False


  # MP Iteration

  results, final_x, phys_e, phys_e2, physical_sol, M_final, M_func1, M_func2, M_func3, M_func4 = MP_iter_woJ(mesh, xi, eta, W, V, x, N, MPIter, epsilon,\
                                                                          plot_phys = False, plot_mesh = False,\
                                                                          rtol = rtol, stol = stol, atol = atol, maxit = maxit, plotsM = plotsM)


  # save progress

  all_results.append([epsilon, results])


  # save the x
  x_list.append(final_x)

  # save errors from physical problem
  physical_l2_e.append(phys_e)
  physical_grad_e.append(phys_e2)


  # SHADOW MESH TEST

  # create shadow mesh

  # shadow = UnitSquareMesh(N,N, quadrilateral = quadrilateral, diagonal = diagonal)
  shadow = RectangleMesh(N, N, 0.25, 1, quadrilateral = quadrilateral, diagonal = diagonal)

  VOM = VertexOnlyMesh(mesh,vertexcoords=mesh.coordinates.dat.data, redundant=False)
  VOM_V = VectorFunctionSpace(VOM,"DG",0)
  VOM_V_IO = VectorFunctionSpace(VOM.input_ordering,"DG",0)

  test_VOM = Function(VOM_V).interpolate(final_x)
  test_VOM_IO = Function(VOM_V_IO).interpolate(test_VOM)

  shadow.coordinates.dat.data[:] = test_VOM_IO.dat.data[:]


  # now look at u on this mesh

  shadowV = FunctionSpace(shadow, "CG", 1)
  u_shadow = Function(shadowV)
  u_shadow.dat.data[:] = physical_sol.dat.data[:]

  # Lets put the exact solution on this mesh

  sol_known = Function(W).interpolate(known_solution(final_x, epsilon))
  shadow_sol = Function(shadowV)
  shadow_sol.dat.data[:] = sol_known.dat.data[:]


  # their difference

  # sol_diff = Function(W).interpolate(sol_known - physical_sol)
  # shadow_diff = Function(shadowV)
  # shadow_diff.dat.data[:] = sol_diff.dat.data[:]

  # # also put M on this mesh

  M_shadow1 = Function(shadowV)
  M_shadow1.dat.data[:] = M_func1.dat.data[:]

  M_shadow2 = Function(shadowV)
  M_shadow2.dat.data[:] = M_func2.dat.data[:]

  M_shadow3 = Function(shadowV)
  M_shadow3.dat.data[:] = M_func3.dat.data[:]

  M_shadow4 = Function(shadowV)
  M_shadow4.dat.data[:] = M_func4.dat.data[:]


  # plot the results
  fig, axes = plt.subplots(2, 2, figsize=(18, 18))

  u_shadow_array = u_shadow.dat.data
  shadow_sol_array = shadow_sol.dat.data
  sol_known_array = sol_known.dat.data
  physical_sol_array = physical_sol.dat.data


  # Interpolate mesh density functions onto a rectangle mesh

  rect_mesh = RectangleMesh(int(N/4), N, 0.25, 1)
  rectV = FunctionSpace(rect_mesh, "CG", 1)
  M_func1_rect = Function(rectV).interpolate(M_func1)
  M_func2_rect = Function(rectV).interpolate(M_func2)
  M_func3_rect = Function(rectV).interpolate(M_func3)
  M_func4_rect = Function(rectV).interpolate(M_func4)


  # small shadow mesh

  small_shadow = RectangleMesh(int(N/2), N, 0.25, 1)
  small_shad_V = FunctionSpace(small_shadow, "CG", 1)
  Mshad1_rect = Function(small_shad_V).interpolate(M_shadow1)
  Mshad2_rect = Function(small_shad_V).interpolate(M_shadow2)
  Mshad3_rect = Function(small_shad_V).interpolate(M_shadow3)
  Mshad4_rect = Function(small_shad_V).interpolate(M_shadow4)


  # sol_diff_array = sol_diff.dat.data
  # shadow_diff_array = shadow_diff.dat.data

  M_shadow1_array = M_shadow1.dat.data
  M_func1_array = M_func1.dat.data
  M_shadow2_array = M_shadow2.dat.data
  M_func2_array = M_func2.dat.data
  M_func3_array = M_func3.dat.data
  M_func4_array = M_func4.dat.data
  M_shadow3_array = M_shadow3.dat.data
  M_shadow4_array = M_shadow4.dat.data


  # Calculate common min and max for the first two plots (for colour bar)
  vmin1 = min(np.min(u_shadow_array), np.min(physical_sol_array))
  vmax1 = max(np.max(u_shadow_array), np.max(physical_sol_array))
  # Calculate common min and max for the first two plots (for colour bar)
  vminKS = min(np.min(shadow_sol_array), np.min(sol_known_array))
  vmaxKS = max(np.max(shadow_sol_array), np.max(sol_known_array))
  # # Calculate common min and max for the bottom two plots (for colour bar)
  # vmin2 = min(np.min(shadow_diff_array), np.min(sol_diff_array))
  # vmax2 = max(np.max(shadow_diff_array), np.max(sol_diff_array))



  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin2 = min(np.min(M_shadow1_array), np.min(M_func1_array))
  vmax2 = max(np.max(M_shadow1_array), np.max(M_func1_array))
  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin3 = min(np.min(M_shadow2_array), np.min(M_func2_array))
  vmax3 = max(np.max(M_shadow2_array), np.max(M_func2_array))
  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin4 = min(np.min(M_shadow3_array), np.min(M_func3_array))
  vmax4 = max(np.max(M_shadow3_array), np.max(M_func3_array))
  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin5 = min(np.min(M_shadow4_array), np.min(M_func4_array))
  vmax5 = max(np.max(M_shadow4_array), np.max(M_func4_array))


  # Plot the shadow mesh solution on the first subplot
  c_shad = tricontourf(u_shadow, axes=axes[0,0], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
  m = triplot(shadow, axes = axes[0,0])
  axes[0,0].set_title('Shadow Mesh Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0,0].set_xlabel(r"$x$")
  axes[0,0].set_ylabel(r"$y$")
  axes[0,0].set_aspect('equal')
  plt.colorbar(c_shad, ax=axes[0,0])
  # Plot the physical solution on the second subplot
  c = tricontourf(physical_sol, axes=axes[0,1], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
  m = triplot(mesh, axes = axes[0,1])
  axes[0,1].set_title('Approx Physical Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0,1].set_xlabel(r"$\xi$")
  axes[0,1].set_ylabel(r"$\eta$")
  axes[0,1].set_aspect('equal')
  plt.colorbar(c, ax=axes[0,1])

  # Plot the shadow mesh known solution
  c_shad = tricontourf(shadow_sol, axes=axes[1,0], cmap = custom_cmap, vmin = vminKS, vmax = vmaxKS)
  m = triplot(shadow, axes = axes[1,0])
  axes[1,0].set_title('(Shadow) Known Physical Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,0].set_xlabel(r"$x$")
  axes[1,0].set_ylabel(r"$y$")
  axes[1,0].set_aspect('equal')
  plt.colorbar(c_shad, ax=axes[1,0])
  # Plot the known solution
  c = tricontourf(sol_known, axes=axes[1,1], cmap = custom_cmap, vmin = vminKS, vmax = vmaxKS)
  m = triplot(mesh, axes = axes[1,1])
  axes[1,1].set_title('Known Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,1].set_xlabel(r"$\xi$")
  axes[1,1].set_ylabel(r"$\eta$")
  axes[1,1].set_aspect('equal')
  plt.colorbar(c, ax=axes[1,1])

  # Save the plots
  plt.tight_layout()
  plt.close(fig)
  figures.append(fig)


  # just take the m00 plots

  fig, axes = plt.subplots(1, 2, figsize=(14, 6))

  # Plot m0 on shadow mesh
  cm_shad = tricontourf(Mshad1_rect, axes=axes[0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  m = triplot(small_shadow, axes = axes[0])
  axes[0].set_title('m00 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0].set_xlabel(r"$x$")
  axes[0].set_ylabel(r"$y$")
  #axes[2,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[0])
  # Plot m0 on mesh
  cm = tricontourf(M_func1_rect, axes=axes[1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  m = triplot(rect_mesh, axes = axes[1])
  axes[1].set_title('m00 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1].set_xlabel(r"$\xi$")
  axes[1].set_ylabel(r"$\eta$")
  #axes[2,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[1])

  # save the plots
  plt.tight_layout()
  plt.close(fig)
  fig_for_later.append(fig)


  # # Plot difference on shadow mesh
  # cm_shad = tricontourf(shadow_diff, axes=axes[2,0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  # m = triplot(shadow, axes = axes[2,0])
  # axes[2,0].set_title('(Shadow) Difference Between Known and Exact Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[2,0].set_xlabel(r"$x$")
  # axes[2,0].set_ylabel(r"$y$")
  # axes[2,0].set_aspect('equal')
  # plt.colorbar(cm_shad, ax=axes[2,0])
  # # diff on mesh
  # cm = tricontourf(sol_diff, axes=axes[2,1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  # m = triplot(mesh, axes = axes[2,1])
  # axes[2,1].set_title('Difference Between Known and Exact Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[2,1].set_xlabel(r"$\xi$")
  # axes[2,1].set_ylabel(r"$\eta$")
  # axes[2,1].set_aspect('equal')
  # plt.colorbar(cm, ax=axes[2,1])


  fig, axes = plt.subplots(4, 2, figsize=(20, 20))

  # Plot m00 on shadow mesh
  cm_shad = tricontourf(Mshad1_rect, axes=axes[0, 0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  m = triplot(small_shadow, axes = axes[0,0])
  axes[0, 0].set_title('m00 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0, 0].set_xlabel(r"$x$")
  axes[0, 0].set_ylabel(r"$y$")
  #axes[2,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[0,0])
  # Plot m00 on mesh
  cm = tricontourf(M_func1_rect, axes=axes[0, 1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  m = triplot(rect_mesh, axes = axes[0, 1])
  axes[0, 1].set_title('m00 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0, 1].set_xlabel(r"$\xi$")
  axes[0, 1].set_ylabel(r"$\eta$")
  #axes[2,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[0, 1])


  # Plot m01 on shadow mesh
  cm_shad = tricontourf(Mshad2_rect, axes=axes[1,0], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
  m = triplot(small_shadow, axes = axes[1,0])
  axes[1,0].set_title('m01 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,0].set_xlabel(r"$x$")
  axes[1,0].set_ylabel(r"$y$")
  #axes[3,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[1,0])
  # Plot m01 on mesh
  cm = tricontourf(M_func2_rect, axes=axes[1,1], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
  m = triplot(rect_mesh, axes = axes[1,1])
  axes[1,1].set_title('m01 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,1].set_xlabel(r"$\xi$")
  axes[1,1].set_ylabel(r"$\eta$")
  #axes[3,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[1,1])


  # Plot m10 on shadow mesh
  cm_shad = tricontourf(Mshad3_rect, axes=axes[2,0], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
  m = triplot(small_shadow, axes = axes[2,0])
  axes[2,0].set_title('m10 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[2,0].set_xlabel(r"$x$")
  axes[2,0].set_ylabel(r"$y$")
  #axes[3,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[2,0])
  # Plot m10 on mesh
  cm = tricontourf(M_func3_rect, axes=axes[2,1], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
  m = triplot(rect_mesh, axes = axes[2,1])
  axes[2,1].set_title('m10 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[2,1].set_xlabel(r"$\xi$")
  axes[2,1].set_ylabel(r"$\eta$")
  #axes[3,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[2,1])


  # Plot m11 on shadow mesh
  cm_shad = tricontourf(Mshad4_rect, axes=axes[3,0], cmap = custom_cmap, vmin = vmin5, vmax = vmax5)
  m = triplot(small_shadow, axes = axes[3,0])
  axes[3,0].set_title('m11 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[3,0].set_xlabel(r"$x$")
  axes[3,0].set_ylabel(r"$y$")
  #axes[3,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[3,0])
  # Plot m11 on mesh
  cm = tricontourf(M_func4_rect, axes=axes[3,1], cmap = custom_cmap, vmin = vmin5, vmax = vmax5)
  m = triplot(rect_mesh, axes = axes[3,1])
  axes[3,1].set_title('m11 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[3,1].set_xlabel(r"$\xi$")
  axes[3,1].set_ylabel(r"$\eta$")
  #axes[3,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[3,1])

  # Save the plots
  plt.tight_layout()
  plt.close(fig)
  figures.append(fig)


  # construct x as the new initial guess


  epsilon = epsilon*(9/10)


  x = final_x




# # We want to save the results of the last MP Iteration to a VTK file





# desired_results = all_results[-1] # last epsilon, desired_results[0] is epsilon, desired_results[1] is [ x, phys_sol ] for each MP Iter

# print("NOW SAVING KNOWN SOLUTION ON THE LAST MESH TO VTK FOR EPS = " + str(desired_results[0]))
# print()
# print()

# num_MP = len(desired_results[1])

# print("There are ", len(desired_results[1]), "MP Iters to consider")
# print()
# print()



# for i in range(num_MP):

#   # pull out x solution and physical solution

#   curr_x = desired_results[1][i][0] # retrieve the x of the ith MP Iter
#   curr_sol = desired_results[1][i][1] # retrieve the phys sol

#   # known solution

#   curr_known = Function(W).interpolate(known_solution(curr_x, desired_results[0]))


#   # shadow mesh creation

#   shadow = RectangleMesh(N, N, 0.25, 1, quadrilateral = quadrilateral, diagonal = diagonal)

#   VOM = VertexOnlyMesh(mesh,vertexcoords=mesh.coordinates.dat.data, redundant=False)
#   VOM_V = VectorFunctionSpace(VOM,"DG",0)
#   VOM_V_IO = VectorFunctionSpace(VOM.input_ordering,"DG",0)

#   test_VOM = Function(VOM_V).interpolate(curr_x)
#   test_VOM_IO = Function(VOM_V_IO).interpolate(test_VOM)

#   shadow.coordinates.dat.data[:] = test_VOM_IO.dat.data[:]


#   # now look at u on this mesh

#   shadowV = FunctionSpace(shadow, "CG", 1)
#   sol_shadow = Function(shadowV)
#   sol_shadow.dat.data[:] = curr_sol.dat.data[:] # save the physical solution

#   # save both of these to a VTK File
#   file_path = '/content/drive/My Drive/Madden_plots/Last_Epsilon_Tests/MPIter' + str(i+1) + '_Known_and_Approx_on_ShadowMesh.pvd'
#   outfile = VTKFile(file_path)
#   outfile.write(sol_shadow, time = 1)

#   # Lets put the exact solution in this function
#   sol_shadow.dat.data[:] = curr_known.dat.data[:]
#   outfile.write(sol_shadow, time = 2)




____________________________________________________________ epsilon = 0.5 ____________________________________________________________



-----------------------------------------------------------------------------------------------------------------------------
Nonlinear Mesh Solve:
  0 SNES Function norm 2.351177162673e-03
  1 SNES Function norm 4.097911448548e-05
  2 SNES Function norm 1.299416537086e-08
  3 SNES Function norm 3.548778187065e-15
  Nonlinear firedrake_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 3
-----------------------------------------------------------------------------------------------------------------------------


-----------------------------------------------------------------------------------------------------------------------------
Nonlinear Mesh Solve:
  0 SNES Function norm 1.953407708986e-05
  1 SNES Function norm 1.507606790444e-09
  2 SNES Function norm 3.564156735696e-15
  Nonlinear firedrake_3_ solve converged due to CONVERGE

In [10]:
# display all my figures

i = 1
while i <= len(figures):
  display(figures[i-1]) # physical solution
  display(figures[i]) # components of M
  i = i + 4
  print()
  print()
  print()
  print()
  print()


# figure_i = 0
# for i in range(10):

#   display(figures[figure_i])
#   figure_i = figure_i + 5
#   print()
#   print()
#   print()
#   print()

# display(figures[-1])




Output hidden; open in https://colab.research.google.com to view.

# Scaling the BI's to compare
This is the same code except I add a scaling factor to the BI's

In [11]:
# # NOW SCALE THE BI's



# # Set My Parameters

# epsilon = 0.5 # start at 0.2 if uniform!
# # N = 24
# # diagonal = "right"
# # quadrilateral = False
# x_choice = 0 # 0 = identity function, 1 = map uniform to shishkin, 2 = map shishkin to uniform
# # xi_choice = 0 # 0 = uniform mesh, 1 = shishkin mesh
# # custom_cmap = "GnBu"
# # deg = 1
# # MPIter = 3
# # cont_steps = 25
# # SNES stopping criteria
# # stol = 1e-8
# # rtol = 1e-8
# # atol = 1e-50
# # maxit = 50

# # scaling the BI's?
# scale_BI = True


# # list to save the x's
# x_list_scaled = []


# # # Set xi


# # if xi_choice == 0: # ci is uniform

# #   mesh = UnitSquareMesh(N, N, quadrilateral = quadrilateral, diagonal = diagonal)
# #   xi, eta = SpatialCoordinate(mesh)


# # if xi_choice == 1: # xi is shishkin

# #   epsm = 0.025
# #   beta = 0.99/np.sqrt(2)
# #   tau_a = 1/2
# #   tau_b = 2*epsm*(1/beta)*np.log(N)
# #   tau = min(tau_a, tau_b)

# #   hB = 2*tau / N
# #   hI = 2*(1-tau) / N


# #   part1 = np.arange(0, tau, hB)
# #   part2 = np.arange(tau, 1.0+hI, hI)

# #   # w_x = np.concatenate((part1, part2[:-1]))
# #   # w_y = np.concatenate((part1, part2[:-1]))
# #   w_x = np.concatenate((part1, part2))
# #   w_y = np.concatenate((part1, part2))

# #   mesh = TensorRectangleMesh(w_x, w_y, quadrilateral = quadrilateral, diagonal = diagonal)
# #   xi, eta = SpatialCoordinate(mesh)



# # # function spaces

# # W = FunctionSpace(mesh, "CG", deg)
# # V = VectorFunctionSpace(mesh, "CG", deg)


# # set x


# if x_choice == 0: # x is identity function

#   x = Function(V).interpolate(as_vector([xi, eta]))

# elif x_choice == 1: # x maps uniform to shihskin

#   epsx = 0.025
#   beta = 0.99/np.sqrt(2)
#   tau_a = 1/2
#   tau_b = 2*epsx*(1/beta)*np.log(N)
#   tau = min(tau_a, tau_b)

#   x_shish = conditional( xi < 0.5, 2*xi*tau, tau + 2*(1-tau)*(xi-1/2) )
#   y_shish = conditional( eta < 0.5, 2*eta*tau, tau + 2*(1-tau)*(eta-1/2) )

#   x = Function(V).interpolate(as_vector([x_shish, y_shish]))


# elif x_choice == 2: # x maps shihskin to uniform

#   x_shish = conditional( xi <= tau, (1/(2*tau))*xi, (1/(2*(1-tau)))*( xi + 1 - 2*tau) )
#   y_shish = conditional( eta < tau, (1/(2*tau))*eta, (1/(2*(1-tau)))*( eta + 1 - 2*tau))

#   x = Function(V).interpolate(as_vector([x_shish, y_shish]))


# # evaluate our starting x function so that we can visualize

# # solution data
# old_mesh_vals = mesh.coordinates.dat.data
# xsol = np.array(x.at(old_mesh_vals))[:,0]
# ysol = np.array(x.at(old_mesh_vals))[:,1]

# # create mesh
# initial_phys_mesh = UnitSquareMesh(N, N, diagonal  = diagonal, quadrilateral = quadrilateral)
# nx = len(xsol)
# new_mesh_vals = np.zeros((nx, 2))
# new_mesh_vals[:,0] = xsol
# new_mesh_vals[:,1] = ysol
# initial_phys_mesh.coordinates.dat.data[:] = new_mesh_vals


# # plot computational and physical mesh

# fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# # ccomputational grid
# xi_plot = triplot(mesh, axes=axes[0])
# axes[0].set_title('Computational Grid')
# axes[0].set_xlabel(r"$\xi$")
# axes[0].set_ylabel(r"$\eta$")
# axes[0].set_aspect('equal')
# axes[0].legend()

# # physical grid
# x_plot = triplot(initial_phys_mesh, axes=axes[1])
# axes[1].set_title('Initial Guess For x')
# axes[1].set_xlabel(r"$x$")
# axes[1].set_ylabel(r"$y$")
# axes[1].set_aspect('equal')
# axes[1].legend()
# plt.tight_layout()
# plt.show()

# print()
# print()

# #physical error list

# physical_l2_e = []
# physical_grad_e = []
# figures = []





# # Set the epsilon continuation + MP solves




# for i in range(cont_steps):


#   physical_l2_e.append(epsilon)
#   physical_grad_e.append(epsilon)


#   print("_"*60 + " epsilon = " + str(epsilon) + " " + "_"*60 )
#   print()
#   print()
#   print()


#   # MP Iteration

#   final_x, phys_e, phys_e2, physical_sol, M_final, M_func1, M_func2 = MP_iter_woJ(mesh, xi, eta, W, V, x, N, MPIter, epsilon,\
#                                                                           plot_phys = False, plot_mesh = False,\
#                                                                           rtol = rtol, stol = stol, atol = atol, maxit = maxit, scale_BI = scale_BI)


#   # save the x
#   x_list_scaled.append(final_x)

#   # save errors from physical problem
#   physical_l2_e.append(phys_e)
#   physical_grad_e.append(phys_e2)


#   # SHADOW MESH TEST

#   # create shadow mesh

#   shadow = UnitSquareMesh(N,N, quadrilateral = quadrilateral, diagonal = diagonal)

#   VOM = VertexOnlyMesh(mesh,vertexcoords=mesh.coordinates.dat.data, redundant=False)
#   VOM_V = VectorFunctionSpace(VOM,"DG",0)
#   VOM_V_IO = VectorFunctionSpace(VOM.input_ordering,"DG",0)

#   test_VOM = Function(VOM_V).interpolate(final_x)
#   test_VOM_IO = Function(VOM_V_IO).interpolate(test_VOM)

#   shadow.coordinates.dat.data[:] = test_VOM_IO.dat.data[:]


#   # now look at u on this mesh

#   shadowV = FunctionSpace(shadow, "CG", 1)
#   u_shadow = Function(shadowV)
#   u_shadow.dat.data[:] = physical_sol.dat.data[:]

#   # Lets put the exact solution on this mesh

#   sol_known = Function(W).interpolate(known_solution(final_x, epsilon))
#   shadow_sol = Function(shadowV)
#   shadow_sol.dat.data[:] = sol_known.dat.data[:]


#   # their difference

#   # sol_diff = Function(W).interpolate(sol_known - physical_sol)
#   # shadow_diff = Function(shadowV)
#   # shadow_diff.dat.data[:] = sol_diff.dat.data[:]

#   # # also put M on this mesh

#   M_shadow1 = Function(shadowV)
#   M_shadow1.dat.data[:] = M_func1.dat.data[:]

#   M_shadow2 = Function(shadowV)
#   M_shadow2.dat.data[:] = M_func2.dat.data[:]


#   # plot the results
#   fig, axes = plt.subplots(4, 2, figsize=(18, 18))

#   u_shadow_array = u_shadow.dat.data
#   shadow_sol_array = shadow_sol.dat.data
#   sol_known_array = sol_known.dat.data
#   physical_sol_array = physical_sol.dat.data

#   # sol_diff_array = sol_diff.dat.data
#   # shadow_diff_array = shadow_diff.dat.data

#   M_shadow1_array = M_shadow1.dat.data
#   M_func1_array = M_func1.dat.data
#   M_shadow2_array = M_shadow2.dat.data
#   M_func2_array = M_func2.dat.data


#   # Calculate common min and max for the first two plots (for colour bar)
#   vmin1 = min(np.min(u_shadow_array), np.min(physical_sol_array))
#   vmax1 = max(np.max(u_shadow_array), np.max(physical_sol_array))
#   # Calculate common min and max for the first two plots (for colour bar)
  # vmin4 = min(np.min(shadow_sol_array), np.min(sol_known_array))
  # vmax4 = max(np.max(shadow_sol_array), np.max(sol_known_array))
  # # # Calculate common min and max for the bottom two plots (for colour bar)
  # # vmin2 = min(np.min(shadow_diff_array), np.min(sol_diff_array))
  # # vmax2 = max(np.max(shadow_diff_array), np.max(sol_diff_array))



  # # Calculate common min and max for the bottom two plots (for colour bar)
  # vmin2 = min(np.min(M_shadow1_array), np.min(M_func1_array))
  # vmax2 = max(np.max(M_shadow1_array), np.max(M_func1_array))
  # # Calculate common min and max for the bottom two plots (for colour bar)
  # vmin3 = min(np.min(M_shadow2_array), np.min(M_func2_array))
  # vmax3 = max(np.max(M_shadow2_array), np.max(M_func2_array))


  # # Plot the shadow mesh solution on the first subplot
  # c_shad = tricontourf(u_shadow, axes=axes[0,0], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
  # m = triplot(shadow, axes = axes[0,0])
  # axes[0,0].set_title('Shadow Mesh Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[0,0].set_xlabel(r"$x$")
  # axes[0,0].set_ylabel(r"$y$")
  # axes[0,0].set_aspect('equal')
  # plt.colorbar(c_shad, ax=axes[0,0])
  # # Plot the physical solution on the second subplot
  # c = tricontourf(physical_sol, axes=axes[0,1], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
  # m = triplot(mesh, axes = axes[0,1])
  # axes[0,1].set_title('Approx Physical Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[0,1].set_xlabel(r"$\xi$")
  # axes[0,1].set_ylabel(r"$\eta$")
  # axes[0,1].set_aspect('equal')
  # plt.colorbar(c, ax=axes[0,1])

  # # Plot the shadow mesh known solution
  # c_shad = tricontourf(shadow_sol, axes=axes[1,0], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
  # m = triplot(shadow, axes = axes[1,0])
  # axes[1,0].set_title('(Shadow) Known Physical Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[1,0].set_xlabel(r"$x$")
  # axes[1,0].set_ylabel(r"$y$")
  # axes[1,0].set_aspect('equal')
  # plt.colorbar(c_shad, ax=axes[1,0])
  # # Plot the known solution
  # c = tricontourf(sol_known, axes=axes[1,1], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
  # m = triplot(mesh, axes = axes[1,1])
  # axes[1,1].set_title('Known Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[1,1].set_xlabel(r"$\xi$")
  # axes[1,1].set_ylabel(r"$\eta$")
  # axes[1,1].set_aspect('equal')
  # plt.colorbar(c, ax=axes[1,1])


  # # # Plot difference on shadow mesh
  # # cm_shad = tricontourf(shadow_diff, axes=axes[2,0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  # # m = triplot(shadow, axes = axes[2,0])
  # # axes[2,0].set_title('(Shadow) Difference Between Known and Exact Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # # axes[2,0].set_xlabel(r"$x$")
  # # axes[2,0].set_ylabel(r"$y$")
  # # axes[2,0].set_aspect('equal')
  # # plt.colorbar(cm_shad, ax=axes[2,0])
  # # # diff on mesh
  # # cm = tricontourf(sol_diff, axes=axes[2,1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  # # m = triplot(mesh, axes = axes[2,1])
  # # axes[2,1].set_title('Difference Between Known and Exact Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # # axes[2,1].set_xlabel(r"$\xi$")
  # # axes[2,1].set_ylabel(r"$\eta$")
  # # axes[2,1].set_aspect('equal')
  # # plt.colorbar(cm, ax=axes[2,1])


  # # Plot m0 on shadow mesh
  # cm_shad = tricontourf(M_shadow1, axes=axes[2,0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  # m = triplot(shadow, axes = axes[2,0])
  # axes[2,0].set_title('m0 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[2,0].set_xlabel(r"$x$")
  # axes[2,0].set_ylabel(r"$y$")
  # axes[2,0].set_aspect('equal')
  # plt.colorbar(cm_shad, ax=axes[2,0])
  # # Plot m0 on mesh
  # cm = tricontourf(M_func1, axes=axes[2,1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  # m = triplot(mesh, axes = axes[2,1])
  # axes[2,1].set_title('m0 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[2,1].set_xlabel(r"$\xi$")
  # axes[2,1].set_ylabel(r"$\eta$")
  # axes[2,1].set_aspect('equal')
  # plt.colorbar(cm, ax=axes[2,1])


  # # Plot m1 on shadow mesh
  # cm_shad = tricontourf(M_shadow2, axes=axes[3,0], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
  # m = triplot(shadow, axes = axes[3,0])
  # axes[3,0].set_title('m1 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[3,0].set_xlabel(r"$x$")
  # axes[3,0].set_ylabel(r"$y$")
  # axes[3,0].set_aspect('equal')
  # plt.colorbar(cm_shad, ax=axes[3,0])
  # # Plot m1 on mesh
  # cm = tricontourf(M_func2, axes=axes[3,1], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
  # m = triplot(mesh, axes = axes[3,1])
  # axes[3,1].set_title('m1 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  # axes[3,1].set_xlabel(r"$\xi$")
  # axes[3,1].set_ylabel(r"$\eta$")
  # axes[3,1].set_aspect('equal')
  # plt.colorbar(cm, ax=axes[3,1])


  # # Show the plots
  # plt.tight_layout()
  # plt.close(fig)
  # figures.append(fig)

  # print('Length of figures', len(figures))
  # print()


  # # construct x as the new initial guess


  # epsilon = epsilon*(9/10)


  # x = final_x

In [12]:
# # print norms of these x's

# for i in range(len(x_list)):
#   print(norm(x_list[i] - x_list_scaled[i]))
#   print()

# Double the Number of Points
Same code except I double the number of points

In [13]:
# # Set My Parameters

# epsilon = 0.5 # start at 0.2 if uniform!
# N = 49
# diagonal = "right"
# quadrilateral = False
# x_choice = 0 # 0 = identity function, 1 = map uniform to shishkin, 2 = map shishkin to uniform
# xi_choice = 0 # 0 = uniform mesh, 1 = shishkin mesh
# custom_cmap = "GnBu"
# deg = 1
# MPIter = 3
# cont_steps = 25
# # SNES stopping criteria
# stol = 1e-8
# rtol = 1e-8
# atol = 1e-50
# maxit = 50

# # scaling the BI's?
# scale_BI = False


# # list to save the x's
# x_list = []


# # Set xi


# if xi_choice == 0: # ci is uniform

#   mesh = UnitSquareMesh(N, N, quadrilateral = quadrilateral, diagonal = diagonal)
#   xi, eta = SpatialCoordinate(mesh)


# if xi_choice == 1: # xi is shishkin

#   epsm = 0.025
#   beta = 0.99/np.sqrt(2)
#   tau_a = 1/2
#   tau_b = 2*epsm*(1/beta)*np.log(N)
#   tau = min(tau_a, tau_b)

#   hB = 2*tau / N
#   hI = 2*(1-tau) / N


#   part1 = np.arange(0, tau, hB)
#   part2 = np.arange(tau, 1.0+hI, hI)

#   # w_x = np.concatenate((part1, part2[:-1]))
#   # w_y = np.concatenate((part1, part2[:-1]))
#   w_x = np.concatenate((part1, part2))
#   w_y = np.concatenate((part1, part2))

#   mesh = TensorRectangleMesh(w_x, w_y, quadrilateral = quadrilateral, diagonal = diagonal)
#   xi, eta = SpatialCoordinate(mesh)



# # function spaces

# W = FunctionSpace(mesh, "CG", deg)
# V = VectorFunctionSpace(mesh, "CG", deg)


# # set x


# if x_choice == 0: # x is identity function

#   x = Function(V).interpolate(as_vector([xi, eta]))

# elif x_choice == 1: # x maps uniform to shihskin

#   epsx = 0.025
#   beta = 0.99/np.sqrt(2)
#   tau_a = 1/2
#   tau_b = 2*epsx*(1/beta)*np.log(N)
#   tau = min(tau_a, tau_b)

#   x_shish = conditional( xi < 0.5, 2*xi*tau, tau + 2*(1-tau)*(xi-1/2) )
#   y_shish = conditional( eta < 0.5, 2*eta*tau, tau + 2*(1-tau)*(eta-1/2) )

#   x = Function(V).interpolate(as_vector([x_shish, y_shish]))


# elif x_choice == 2: # x maps shihskin to uniform

#   x_shish = conditional( xi <= tau, (1/(2*tau))*xi, (1/(2*(1-tau)))*( xi + 1 - 2*tau) )
#   y_shish = conditional( eta < tau, (1/(2*tau))*eta, (1/(2*(1-tau)))*( eta + 1 - 2*tau))

#   x = Function(V).interpolate(as_vector([x_shish, y_shish]))


# # evaluate our starting x function so that we can visualize

# # solution data
# old_mesh_vals = mesh.coordinates.dat.data
# xsol = np.array(x.at(old_mesh_vals))[:,0]
# ysol = np.array(x.at(old_mesh_vals))[:,1]

# # create mesh
# initial_phys_mesh = UnitSquareMesh(N, N, diagonal  = diagonal, quadrilateral = quadrilateral)
# nx = len(xsol)
# new_mesh_vals = np.zeros((nx, 2))
# new_mesh_vals[:,0] = xsol
# new_mesh_vals[:,1] = ysol
# initial_phys_mesh.coordinates.dat.data[:] = new_mesh_vals


# # plot computational and physical mesh

# fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# # ccomputational grid
# xi_plot = triplot(mesh, axes=axes[0])
# axes[0].set_title('Computational Grid')
# axes[0].set_xlabel(r"$\xi$")
# axes[0].set_ylabel(r"$\eta$")
# axes[0].set_aspect('equal')
# axes[0].legend()

# # physical grid
# x_plot = triplot(initial_phys_mesh, axes=axes[1])
# axes[1].set_title('Initial Guess For x')
# axes[1].set_xlabel(r"$x$")
# axes[1].set_ylabel(r"$y$")
# axes[1].set_aspect('equal')
# axes[1].legend()
# plt.tight_layout()
# plt.show()

# print()
# print()

# #physical error list

# physical_l2_e = []
# physical_grad_e = []
# figures_double = []





# # Set the epsilon continuation + MP solves




# for i in range(cont_steps):


#   physical_l2_e.append(epsilon)
#   physical_grad_e.append(epsilon)


#   print("_"*60 + " epsilon = " + str(epsilon) + " " + "_"*60 )
#   print()
#   print()
#   print()


#   # Do I want to plot the starting physical solve and M's zoomed in?

#   if i == 0:
#     plotsM = True
#   else:
#     plotsM = False


#   # MP Iteration

#   final_x, phys_e, phys_e2, physical_sol, M_final, M_func1, M_func2, M_func3, M_func4 = MP_iter_woJ(mesh, xi, eta, W, V, x, N, MPIter, epsilon,\
#                                                                           plot_phys = False, plot_mesh = False,\
#                                                                           rtol = rtol, stol = stol, atol = atol, maxit = maxit, plotsM = plotsM)


#   # save the x
#   x_list.append(final_x)

#   # save errors from physical problem
#   physical_l2_e.append(phys_e)
#   physical_grad_e.append(phys_e2)


#   # SHADOW MESH TEST

#   # create shadow mesh

#   # shadow = UnitSquareMesh(N,N, quadrilateral = quadrilateral, diagonal = diagonal)
#   shadow = RectangleMesh(N, N, 0.25, 1, quadrilateral = quadrilateral, diagonal = diagonal)

#   VOM = VertexOnlyMesh(mesh,vertexcoords=mesh.coordinates.dat.data, redundant=False)
#   VOM_V = VectorFunctionSpace(VOM,"DG",0)
#   VOM_V_IO = VectorFunctionSpace(VOM.input_ordering,"DG",0)

#   test_VOM = Function(VOM_V).interpolate(final_x)
#   test_VOM_IO = Function(VOM_V_IO).interpolate(test_VOM)

#   shadow.coordinates.dat.data[:] = test_VOM_IO.dat.data[:]


#   # now look at u on this mesh

#   shadowV = FunctionSpace(shadow, "CG", 1)
#   u_shadow = Function(shadowV)
#   u_shadow.dat.data[:] = physical_sol.dat.data[:]

#   # Lets put the exact solution on this mesh

#   sol_known = Function(W).interpolate(known_solution(final_x, epsilon))
#   shadow_sol = Function(shadowV)
#   shadow_sol.dat.data[:] = sol_known.dat.data[:]


#   # # also put M on this mesh

#   M_shadow1 = Function(shadowV)
#   M_shadow1.dat.data[:] = M_func1.dat.data[:]

#   M_shadow2 = Function(shadowV)
#   M_shadow2.dat.data[:] = M_func2.dat.data[:]



#   # plot the results
#   fig, axes = plt.subplots(2, 2, figsize=(18, 10))

#   u_shadow_array = u_shadow.dat.data
#   shadow_sol_array = shadow_sol.dat.data
#   sol_known_array = sol_known.dat.data
#   physical_sol_array = physical_sol.dat.data


#   # Interpolate mesh density functions onto a rectangle mesh

#   rect_mesh = RectangleMesh(int(N/4), N, 0.25, 1)
#   rectV = FunctionSpace(rect_mesh, "CG", 1)
#   M_func1_rect = Function(rectV).interpolate(M_func1)
#   M_func2_rect = Function(rectV).interpolate(M_func2)


#   # small shadow mesh

#   small_shadow = RectangleMesh(int(N/2), N, 0.25, 1)
#   small_shad_V = FunctionSpace(small_shadow, "CG", 1)
#   Mshad1_rect = Function(small_shad_V).interpolate(M_shadow1)
#   Mshad2_rect = Function(small_shad_V).interpolate(M_shadow2)


#   # sol_diff_array = sol_diff.dat.data
#   # shadow_diff_array = shadow_diff.dat.data

#   M_shadow1_array = M_shadow1.dat.data
#   M_func1_array = M_func1.dat.data
#   M_shadow2_array = M_shadow2.dat.data
#   M_func2_array = M_func2.dat.data


#   # Calculate common min and max for the first two plots (for colour bar)
#   vmin1 = min(np.min(u_shadow_array), np.min(physical_sol_array))
#   vmax1 = max(np.max(u_shadow_array), np.max(physical_sol_array))
#   # Calculate common min and max for the first two plots (for colour bar)
#   vmin4 = min(np.min(shadow_sol_array), np.min(sol_known_array))
#   vmax4 = max(np.max(shadow_sol_array), np.max(sol_known_array))
#   # # Calculate common min and max for the bottom two plots (for colour bar)
#   # vmin2 = min(np.min(shadow_diff_array), np.min(sol_diff_array))
#   # vmax2 = max(np.max(shadow_diff_array), np.max(sol_diff_array))



#   # Calculate common min and max for the bottom two plots (for colour bar)
#   vmin2 = min(np.min(M_shadow1_array), np.min(M_func1_array))
#   vmax2 = max(np.max(M_shadow1_array), np.max(M_func1_array))
#   # Calculate common min and max for the bottom two plots (for colour bar)
#   vmin3 = min(np.min(M_shadow2_array), np.min(M_func2_array))
#   vmax3 = max(np.max(M_shadow2_array), np.max(M_func2_array))


#   # Plot the shadow mesh solution on the first subplot
#   c_shad = tricontourf(u_shadow, axes=axes[0,0], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
#   m = triplot(shadow, axes = axes[0,0])
#   axes[0,0].set_title('Shadow Mesh Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   axes[0,0].set_xlabel(r"$x$")
#   axes[0,0].set_ylabel(r"$y$")
#   axes[0,0].set_aspect('equal')
#   plt.colorbar(c_shad, ax=axes[0,0])
#   # Plot the physical solution on the second subplot
#   c = tricontourf(physical_sol, axes=axes[0,1], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
#   m = triplot(mesh, axes = axes[0,1])
#   axes[0,1].set_title('Approx Physical Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   axes[0,1].set_xlabel(r"$\xi$")
#   axes[0,1].set_ylabel(r"$\eta$")
#   axes[0,1].set_aspect('equal')
#   plt.colorbar(c, ax=axes[0,1])

#   # Plot the shadow mesh known solution
#   c_shad = tricontourf(shadow_sol, axes=axes[1,0], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
#   m = triplot(shadow, axes = axes[1,0])
#   axes[1,0].set_title('(Shadow) Known Physical Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   axes[1,0].set_xlabel(r"$x$")
#   axes[1,0].set_ylabel(r"$y$")
#   axes[1,0].set_aspect('equal')
#   plt.colorbar(c_shad, ax=axes[1,0])
#   # Plot the known solution
#   c = tricontourf(sol_known, axes=axes[1,1], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
#   m = triplot(mesh, axes = axes[1,1])
#   axes[1,1].set_title('Known Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   axes[1,1].set_xlabel(r"$\xi$")
#   axes[1,1].set_ylabel(r"$\eta$")
#   axes[1,1].set_aspect('equal')
#   plt.colorbar(c, ax=axes[1,1])

#   # Show the plots
#   plt.tight_layout()
#   plt.close(fig)
#   figures_double.append(fig)


#   # # Plot difference on shadow mesh
#   # cm_shad = tricontourf(shadow_diff, axes=axes[2,0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
#   # m = triplot(shadow, axes = axes[2,0])
#   # axes[2,0].set_title('(Shadow) Difference Between Known and Exact Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   # axes[2,0].set_xlabel(r"$x$")
#   # axes[2,0].set_ylabel(r"$y$")
#   # axes[2,0].set_aspect('equal')
#   # plt.colorbar(cm_shad, ax=axes[2,0])
#   # # diff on mesh
#   # cm = tricontourf(sol_diff, axes=axes[2,1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
#   # m = triplot(mesh, axes = axes[2,1])
#   # axes[2,1].set_title('Difference Between Known and Exact Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   # axes[2,1].set_xlabel(r"$\xi$")
#   # axes[2,1].set_ylabel(r"$\eta$")
#   # axes[2,1].set_aspect('equal')
#   # plt.colorbar(cm, ax=axes[2,1])


#   fig, axes = plt.subplots(1, 2, figsize=(14, 6))

#   # Plot m00 on shadow mesh
#   cm_shad = tricontourf(Mshad1_rect, axes=axes[0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
#   m = triplot(small_shadow, axes = axes[0])
#   axes[0].set_title('m0 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   axes[0].set_xlabel(r"$x$")
#   axes[0].set_ylabel(r"$y$")
#   #axes[2,0].set_aspect('equal')
#   plt.colorbar(cm_shad, ax=axes[0])
#   # Plot m00 on mesh
#   cm = tricontourf(M_func1_rect, axes=axes[1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
#   m = triplot(rect_mesh, axes = axes[1])
#   axes[1].set_title('m0 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   axes[1].set_xlabel(r"$\xi$")
#   axes[1].set_ylabel(r"$\eta$")
#   #axes[2,1].set_aspect('equal')
#   plt.colorbar(cm, ax=axes[1])


#   # # Plot m1 on shadow mesh
#   # cm_shad = tricontourf(Mshad2_rect, axes=axes[1,0], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
#   # m = triplot(small_shadow, axes = axes[1,0])
#   # axes[1,0].set_title('m1 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   # axes[1,0].set_xlabel(r"$x$")
#   # axes[1,0].set_ylabel(r"$y$")
#   # #axes[3,0].set_aspect('equal')
#   # plt.colorbar(cm_shad, ax=axes[1,0])
#   # # Plot m1 on mesh
#   # cm = tricontourf(M_func2_rect, axes=axes[1,1], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
#   # m = triplot(rect_mesh, axes = axes[1,1])
#   # axes[1,1].set_title('m1 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
#   # axes[1,1].set_xlabel(r"$\xi$")
#   # axes[1,1].set_ylabel(r"$\eta$")
#   # #axes[3,1].set_aspect('equal')
#   # plt.colorbar(cm, ax=axes[1,1])


#   # Save the plots
#   plt.tight_layout()
#   plt.close(fig)
#   figures_double.append(fig)


#   # construct x as the new initial guess


#   epsilon = epsilon*(9/10)


#   x = final_x


In [14]:
# print the comparison




# i = 1
# j = 0
# while i <= len(figures_double):

#   display(fig_for_later[j])
#   display(figures_double[i])
#   print()
#   print()
#   print()
#   print()
#   i += 2 + 5
#   j += 1 + 5

# Error Plots

In [15]:
# # print out L2 errors of the physical solves


# print("  Epsilon             |      L2 Error       ")
# print("_"*175)
# print()

# i = 0
# while i <= (len(physical_l2_e)-1):

#   print("%.8f" %physical_l2_e[i], end = "                     ")
#   print(physical_l2_e[i+1])
#   print("_"*175)

#   i = i+2



In [16]:
# # print out grad errors of the physical solves


# print("  Epsilon             |      Grad Error       ")
# print("_"*175)
# print()

# i = 0
# while i <= (len(physical_grad_e)-1):

#   print("%.8f" %physical_grad_e[i], end = "                     ")
#   print(physical_grad_e[i+1])
#   print("_"*175)

#   i = i+2

In [17]:
# desired_e_l2 = []
# desired_e_grad = []
# eps_list = []


# # pull out last error of each MP Iteration and epsilon list

# i = 0
# while i <= (len(physical_l2_e)-1):

#     eps_list.append(physical_l2_e[i])
#     desired_e_l2.append( physical_l2_e[i+1][-1] )
#     desired_e_grad.append( physical_grad_e[i+1][-1])

#     i = i+2



# # plot epsilon vs error for each error type


# fig, axes = plt.subplots(1, 2, figsize=(14, 6))
# axes[0].plot(np.log10(eps_list), np.log10(desired_e_l2))
# axes[0].set_xlabel(r"$\epsilon$")
# axes[0].set_ylabel("L2 Error")
# # axes[0].set_xscale('symlog', linthresh = 1e-3)
# # axes[0].set_yscale('symlog', linthresh = 1e-3)
# axes[0].set_aspect('equal')
# axes[0].set_title("Log-Log Plot of L2 Error Vs. Epsilon for the Physical Solution")

# axes[1].plot(np.log10(eps_list), np.log10(desired_e_grad))
# axes[1].set_xlabel(r"$\epsilon$")
# axes[1].set_ylabel("Gradient Style Error")
# axes[1].set_title("Log-Log Plot of Gradient Error Vs. Epsilon for the Physical Solution")
# axes[1].set_aspect('equal')

# plt.tight_layout()
# plt.show()


# #print("Data range:", min([abs(desired_e_l2[i]) for i in range(len(desired_e_l2)) ]), max([abs(desired_e_l2[i]) for i in range(len(desired_e_l2)) ]))
